In [9]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
from scipy import stats

In [10]:
tickers = pd.read_csv('top_50_indian_stocks.csv')
tickers.head()

,Ticker,Company Name
0,RELIANCE.NS,Reliance Industries
1,TCS.NS,Tata Consultancy Services
2,HDFCBANK.NS,HDFC Bank
3,INFY.NS,Infosys
4,ICICIBANK.NS,ICICI Bank


## Weighted Scoring Model for finding the final score

In [15]:
def create_dividend_df(tickers):
    columns = [
        "Ticker", 
        "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
        
    ]
    
    dividend_df = pd.DataFrame(columns=columns)
    
    for stock in tickers:
        ticker = yf.Ticker(stock)
        info = ticker.info 
        
        dividend_yield = info.get("dividendYield", np.nan) * 100 if info.get("dividendYield") else np.nan
        dividend_rate = info.get("dividendRate", np.nan) 
        payout_ratio = info.get("payoutRatio", np.nan) * 100 if info.get("payoutRatio") else np.nan
        five_year_avg_dividend_yield = info.get("fiveYearAvgDividendYield", np.nan) * 100 if info.get("fiveYearAvgDividendYield") else np.nan
        earning_growth = info.get("earningsGrowth", np.nan) * 100 if info.get("earningsGrowth") else np.nan
        
        dividend_df.loc[len(dividend_df)] = [stock, dividend_yield, dividend_rate, payout_ratio, five_year_avg_dividend_yield, earning_growth]
        
    ## we have got our df with dividend information
    
    numeric_cols = [
          "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
    ]
    

    for col in numeric_cols:
        if col == "Payout Ratio":
            dividend_df[col+ " Normalised"] = 1 - ((dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min()))
        else:
            dividend_df[col+ " Normalised"] = (dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min())
        
    return dividend_df

In [16]:
tickers_list = tickers["Ticker"].values.tolist()
dividend_df = create_dividend_df(tickers_list)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised
0,RELIANCE.NS,40.0,5.0,9.779999,44.0,7.4,0.029240,0.028777,0.053646,0.041621,0.177521
1,TCS.NS,161.0,58.0,42.310002,127.0,13.0,0.130326,0.410072,0.254697,0.132530,0.187043
2,HDFCBANK.NS,115.0,19.5,21.820000,67.0,1.5,0.091896,0.133094,0.128059,0.066813,0.167489
3,INFY.NS,243.0,41.0,62.000000,212.0,7.3,0.198830,0.287770,0.376391,0.225630,0.177351
4,ICICIBANK.NS,82.0,10.0,14.570000,99.0,16.0,0.064327,0.064748,0.083251,0.101862,0.192144
5,HINDUNILVR.NS,195.0,43.0,94.030000,148.0,18.9,0.158730,0.302158,0.574351,0.155531,0.197075
6,SBIN.NS,187.0,13.7,15.410000,139.0,70.3,0.152047,0.091367,0.088443,0.145674,0.284475
7,BAJFINANCE.NS,43.0,36.0,13.960000,24.0,15.7,0.031746,0.251799,0.079481,0.019715,0.191634
8,BHARTIARTL.NS,49.0,8.0,19.430000,80.0,491.1,0.036759,0.050360,0.113288,0.081051,1.000000
9,ITC.NS,347.0,14.0,85.350000,406.0,-7.5,0.285714,0.093525,0.520705,0.438116,0.152185


In [17]:
weights = {
        "Dividend Yield(%) Normalised": 0.3,
        "Dividend Rate Normalised": 0.2,
        "Payout Ratio(%) Normalised": 0.2,
        "5 Year Avg Dividend Yield(%) Normalised": 0.2,
        "Earning Growth(%) Normalised": 0.1
    }

dividend_df["Dividend Score"] = dividend_df[[col for col in weights.keys()]].mul(list(weights.values())).sum(axis = 1)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
0,RELIANCE.NS,40.0,5.0,9.779999,44.0,7.4,0.029240,0.028777,0.053646,0.041621,0.177521,0.051333
1,TCS.NS,161.0,58.0,42.310002,127.0,13.0,0.130326,0.410072,0.254697,0.132530,0.187043,0.217262
2,HDFCBANK.NS,115.0,19.5,21.820000,67.0,1.5,0.091896,0.133094,0.128059,0.066813,0.167489,0.109911
3,INFY.NS,243.0,41.0,62.000000,212.0,7.3,0.198830,0.287770,0.376391,0.225630,0.177351,0.255342
4,ICICIBANK.NS,82.0,10.0,14.570000,99.0,16.0,0.064327,0.064748,0.083251,0.101862,0.192144,0.088485
5,HINDUNILVR.NS,195.0,43.0,94.030000,148.0,18.9,0.158730,0.302158,0.574351,0.155531,0.197075,0.273735
6,SBIN.NS,187.0,13.7,15.410000,139.0,70.3,0.152047,0.091367,0.088443,0.145674,0.284475,0.139158
7,BAJFINANCE.NS,43.0,36.0,13.960000,24.0,15.7,0.031746,0.251799,0.079481,0.019715,0.191634,0.098886
8,BHARTIARTL.NS,49.0,8.0,19.430000,80.0,491.1,0.036759,0.050360,0.113288,0.081051,1.000000,0.159967
9,ITC.NS,347.0,14.0,85.350000,406.0,-7.5,0.285714,0.093525,0.520705,0.438116,0.152185,0.311402


In [18]:
dividend_df =dividend_df.sort_values(by = "Dividend Score", ascending = False)
dividend_df.head(10)

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
45,IOC.NS,1202.0,15.0,90.669996,735.0,-76.5,1.000000,0.100719,0.553585,0.798467,0.034858,0.594040
30,COALINDIA.NS,830.0,31.6,46.680000,919.0,-6.3,0.689223,0.220144,0.281706,1.000000,0.154225,0.522559
42,HEROMOTOCO.NS,383.0,140.0,55.480000,317.0,1.3,0.315789,1.000000,0.336094,0.340635,0.167148,0.446798
20,ONGC.NS,751.0,17.5,40.310000,529.0,-16.8,0.623225,0.118705,0.242336,0.572837,0.136371,0.387380
22,POWERGRID.NS,570.0,15.0,70.519996,515.0,-4.2,0.472013,0.100719,0.429048,0.557503,0.157796,0.374838
31,BPCL.NS,593.0,15.5,32.650003,575.0,19.3,0.491228,0.104317,0.194994,0.623220,0.197755,0.351650
13,HCLTECH.NS,347.0,54.0,85.850000,293.0,5.3,0.285714,0.381295,0.523795,0.314348,0.173950,0.346997
28,TECHM.NS,288.0,43.0,101.870000,225.0,92.4,0.236424,0.302158,0.622806,0.239869,0.322054,0.336099
41,TATASTEEL.NS,238.0,3.6,162.900000,291.0,-38.1,0.194653,0.018705,1.000000,0.312158,0.100153,0.334584
9,ITC.NS,347.0,14.0,85.350000,406.0,-7.5,0.285714,0.093525,0.520705,0.438116,0.152185,0.311402
